<a href="https://colab.research.google.com/github/tomonari-masada/course2023-nlp/blob/main/02_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# テキストデータの前処理

* テキストデータは、長い長い文字列。
* 長い長い文字列のままでは、普通は分析できない。
* 今回は、基本的な前処理について学ぶ。
 * 英語と日本語の両方。

## BoWにおける語彙の重要性
* BoWの範囲内でテキストデータを分析する場合は・・・
* 語彙がそのまま特徴量(feature)になるので・・・
* きれいな語彙集合を作ることが重要。つまり、前処理は重要。

## str型のメソッドによる前処理

* Pythonに元々備わっている機能を使うだけで、簡単な前処理ができる。

 * 問：元のテキストにあった大文字と小文字の区別を無くしてしまうことのメリットとデメリットは何か？

### 大文字から小文字への変換

In [ ]:
text = "The quick brown fox jumped over The Big Dog"
text

In [ ]:
text.lower()

### 空白文字によるテキストの分割

In [ ]:
text.lower().split()

## NLTK (Natural Language Toolkit)

* https://www.nltk.org/

* Pythonで自然言語処理を行うためのライブラリ
  * 2001年スタートらしい。

* WordNetも使える（ここでは説明しない）
 * https://www.nltk.org/howto/wordnet.html
 * WordNetについては『IT Text 自然言語処理の基礎』3.2.2(a)を参照。

In [ ]:
import nltk

### NLTKによるトークン化

* 文に分ける、単語に分ける、など、長い文字列としてのテキストを小さな単位へ分割することを一般にtokenizationと言う。

In [ ]:
text = (
    "Avram Noam Chomsky (born December 7, 1928) is an American professor and public intellectual known for his work in linguistics, political activism, and social criticism. "
    'Sometimes called "the father of modern linguistics", Chomsky is also a major figure in analytic philosophy and one of the founders of the field of cognitive science. '
    "He is a laureate professor of linguistics at the University of Arizona and an institute professor emeritus at the Massachusetts Institute of Technology (MIT)."
)
text

* 文へtokenize
 * `punkt`というパッケージが必要なのでダウンロードしておく。

In [ ]:
nltk.download('punkt')

In [ ]:
nltk.sent_tokenize(text)

* 単語へtokenize

In [ ]:
print(nltk.word_tokenize(text))

## spaCy

* https://spacy.io/
* spaCyも、Pythonの有名な自然言語処理ライブラリ。2015年スタートらしい。



In [ ]:
import spacy

### spaCyの「モデル」
* 言語ごとにモデルが用意されている。
 * https://spacy.io/models
* モデルにはいくつか種類があり、規模が異なる。
 * 大きなモデルでは、かなり複雑な処理ができる。

### Tokenization

* モデルをロードする。
 * `sm`はsmallの意味。

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
doc = nlp(text)

* 文へtokenize

In [ ]:
[sent.text for sent in doc.sents]

* 単語へtokenize

In [ ]:
print([token.text for token in doc])

### POS tagging
* POS = part of speech （品詞）

In [ ]:
print([token.pos_ for token in doc])

## 様々な前処理の実践

### 正規表現による前処理
* 正規表現 = regular expression
* ここでは、特殊文字、数字、記号の除去を正規表現を使っておこなう。

* 問：テキストデータの前処理において特殊文字、数字、記号などを除去することのメリットとデメリットは何か？

In [ ]:
import re

def remove_special_characters(text, remove_digits=False):
  pattern = r'[^a-zA-Z0-9\s]' if not remove_digits else r'[^a-zA-Z\s]'
  text = re.sub(pattern, '', text)
  return text

In [ ]:
s = "Well this was fun! See you at 7:30, What do you think!!? #$@@9318@ 🙂🙂🙂"
s

In [ ]:
remove_special_characters(s, remove_digits=True)

In [ ]:
remove_special_characters(s)

### Lemmatization

* 動詞や形容詞は原型に、名詞は単数形に、等と、単語の元々の形に直すこと。

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")

def lemmatize(text):
  return [word.lemma_ for word in nlp(text)]

In [ ]:
lemmatize('lying')

In [ ]:
lemmatize('goes')

In [ ]:
text = "Avram Noam Chomsky (born December 7, 1928) is an American professor and public intellectual known for his work in linguistics, political activism, and social criticism."
print(lemmatize(text))

### ストップワード

* ストップワードとは、あまりに頻出するため、分析に役に立ちそうにない単語のこと。
 * 「これこそが英語（日本語、等）のストップワードだ！」と言えるような決定的なストップワードのリストがあるわけではない。
 * 主要なNLPライブラリでは、あらかじめ用意されたストップワードのリストを使うことができる。
 * しかし、分析したいテキストデータに合わせて、ストップワードのリストをカスタマイズすることも、よくある。
* **深層学習の世界では、ストップワードの除去は、基本的に行わない。**
 * 深層学習言語モデルでは、元のままのテキストをサブワードへ分割するため。
 * サブワードへの分割を使うと、むしろ、ストップワードの役割を尊重しつつ、テキストを分析できる。
* BoWの範囲内でテキストを分析するときは、普通、ストップワードは除去する。

### spaCyのストップワード
* 活用変化する単語はlemmaの形で登録されている。

In [ ]:
from spacy.lang.en.stop_words import STOP_WORDS

print(STOP_WORDS)
print(len(STOP_WORDS))

In [ ]:
def remove_stopwords(text):
  stopwords = spacy.lang.en.stop_words.STOP_WORDS
  return [token.text for token in nlp(text) if token.lemma_ not in stopwords]

In [ ]:
text = "Avram Noam Chomsky (born December 7, 1928) is an American professor and public intellectual known for his work in linguistics, political activism, and social criticism."
print(remove_stopwords(text))

## 現代的なtokenization

### サブワード(subword)
* 深層学習の世界では、サブワードをテキストの構成要素とみなす。
 * 単語よりもさらに粒度が細かい。
* サブワードを構成要素とすることで、未知語の問題を回避できる。
 * 全ての文字をサブワードとして語彙に登録しておけば、未知語は無くなる。

### 文字・サブワード・単語
 * https://ai.googleblog.com/2020/09/advancing-nlp-with-efficient-projection.html

![Segmentation.png](https://raw.githubusercontent.com/tomonari-masada/course2022-nlp/main/Segmentation.png)
![inherent_task_complexity.png](https://raw.githubusercontent.com/tomonari-masada/course2022-nlp/main/image3.png)

### サブワードへのトークン化
* ここではHugging FaceのTransformersライブラリを使ってみる。

In [ ]:
!pip install transformers[torch]

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
text = "Avram Noam Chomsky (born December 7, 1928) is an American professor and public intellectual known for his work in linguistics, political activism, and social criticism."

In [ ]:
print(tokenizer.tokenize(text))

### テキスト集合からサブワードの語彙を得るアルゴリズム
* Byte-Pair Encoding (BPE) などについて説明がある。
 * https://huggingface.co/docs/transformers/tokenizer_summary

## 日本語テキストの分析

### spaCyの日本語モデル

In [ ]:
!python -m spacy download ja_core_news_sm

In [ ]:
import spacy

In [ ]:
text = (
    "ノーム・チョムスキーは1928年12月7日、アメリカ合衆国ペンシルベニア州フィラデルフィアのイースト・オーク・レーン近郊で生まれた。"
    "父ウィリアム・チョムスキーは当時ロシア帝国支配下のウクライナで生まれたが、戦乱を避けて1913年にアメリカへ渡った。"
    "メリーランド州ボルチモアの搾取工場で働き、貯蓄してジョンズ・ホプキンス大学で学んだ甲斐もあり市のヘブライ人系小学校教師の職を得た。"
)


In [ ]:
nlp = spacy.load("ja_core_news_sm")
doc = nlp(text)
for token in doc:
  print((token.text, token.lemma_, token.pos_))

# 本日の課題
* livedoor ニュースコーパスでの単語の出現回数を調べよう。
 * https://www.rondhuit.com/download.html#news%20corpus
* 活用変化する単語は、原型に戻してから、出現回数を数えよう。

In [ ]:
!wget https://www.rondhuit.com/download/ldcc-20140209.tar.gz

* このデータセットの前処理については、下の記事を参照。
 * https://tech.fusic.co.jp/posts/2021-04-23-bert-multi-classification/

In [ ]:
def read_title(f):
  next(f) # URL
  next(f) # タイムスタンプ
  title = next(f) # 3行目を返す：タイトル
  title = title.decode('utf-8')
  brackets_tail = re.compile('【[^】]*】$')
  brackets_head = re.compile('^【[^】]*】')
  return re.sub(brackets_head, "", re.sub(brackets_tail, "", title))[:-1]

In [ ]:
import tarfile

corpus = []

tar_fname = "ldcc-20140209.tar.gz"

with tarfile.open(tar_fname) as tf:
  for item in tf:
    if "LICENSE.txt" in item.name:
      continue
    if len(item.name.split('/')) < 3:
      continue
    if not item.name.endswith(".txt"):
      continue
    fname = item.name
    # 今回はクラス名は要らない
    #class_name = fname.split('/')[1]
    f = tf.extractfile(fname)
    title = read_title(f)
    corpus.append(title)

In [ ]:
len(corpus)

In [ ]:
nlp = spacy.load("ja_core_news_sm")
print([token.lemma_ for token in nlp(corpus[0])])